In [1]:

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
import anthropic

In [2]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
if not ANTHROPIC_API_KEY:
    logging.error("Please set the ANTHROPIC_API_KEY environment variable.")
    sys.exit(1)

In [5]:
if ANTHROPIC_API_KEY.strip() != ANTHROPIC_API_KEY:
  print("My key contains leading or trailing space")
elif ANTHROPIC_API_KEY[:7] != "sk-ant-":
  print("My key doesn't start sk-ant-")
else:
  print("My key looks ok..")

My key looks ok..


In [6]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [7]:
# ed = Website("https://edwarddonner.com")
# print(ed.title)
# print(ed.text)

## Helper Functions

In [8]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [9]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [10]:
# See how this function creates exactly the format above

def messages_for(website):
    return {"system": system_prompt,
            "messages":[
                        {"role": "user", "content": user_prompt_for(website)}
                ]
            }

## Payload info

In [11]:
ANTHROPIC_API = "https://api.anthropic.com/v1/messages"
HEADERS = {
        "x-api-key": ANTHROPIC_API_KEY,
        "anthropic-version": "2023-06-01",
        "Content-Type": "application/json"
    }
MODEL = "claude-3-5-sonnet-latest"

In [12]:
# print("Payload:", payload)
# print("Headers:", HEADERS)


## First approach

In [13]:
def summarize_requests(url):
    website = Website(url)
    payload = {
        "model": MODEL,
        "max_tokens": 1000,
        "system": system_prompt,  # Add the system prompt as a top-level parameter
        "messages": [
            {"role": "user", "content": user_prompt_for(website)}  # Only include the user message
        ]
    }
    
    try:
        response = requests.post(ANTHROPIC_API, headers=HEADERS, json=payload)
        response.raise_for_status()  # Raise an HTTPError for bad responses
        return response.json()["content"][0]["text"]
    except requests.exceptions.RequestException as e:
        logging.error(f"Request failed: {e}")
        logging.error(f"Response: {response.text if response else 'No response'}")
        raise

In [14]:
def display_summary_requests(url):
    summary = summarize_requests(url)
    display(Markdown(summary))

In [15]:
url = "https://edwarddonner.com"
display_summary_requests(url)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): edwarddonner.com:443
DEBUG:urllib3.connectionpool:https://edwarddonner.com:443 "GET / HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.anthropic.com:443
DEBUG:urllib3.connectionpool:https://api.anthropic.com:443 "POST /v1/messages HTTP/1.1" 200 None


# Summary of Edward Donner's Website

## About Ed
Ed Donner is a tech entrepreneur and AI enthusiast who is currently the co-founder and CTO of Nebula.io. He was previously the founder and CEO of untapt, an AI startup that was acquired in 2021. His interests include:
- Coding
- Experimenting with LLMs
- DJing
- Electronic music production
- Technology discussions

## Professional Work
- Currently leads Nebula.io, which uses AI for talent discovery and recruitment
- Company has developed proprietary LLMs for talent management
- Holds patents for their matching model
- Platform has received awards and media coverage

## Projects
The website features two notable projects:
1. **Connect Four** - (details not provided)
2. **Outsmart** - An arena for LLMs to compete in diplomacy and strategy

## Recent Posts
- The Complete Agentic AI Engineering Course (April 2025)
- LLM Workshop – Hands-on with Agents – resources (January 2025)
- Welcome, SuperDataScientists! (December 2024)
- Mastering AI and LLM Engineering – Resources (November 2024)

The website serves as both a professional portfolio and a platform for sharing knowledge about AI and LLM engineering.

## Anthropic Package approach

In [16]:
client = anthropic.Client(api_key=ANTHROPIC_API_KEY)

url = "https://edwarddonner.com"


In [17]:
def summarize_package(url):
    website = Website(url)
    messages = messages_for(website) 
    response = client.messages.create(model=MODEL, temperature=0.0, max_tokens=500, **messages)
    return response.content[0].text

In [18]:
def display_summary_package(url):
    summary = summarize_package(url)
    display(Markdown(summary))

In [19]:
url = "https://edwarddonner.com"
display_summary_package(url)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): edwarddonner.com:443
DEBUG:urllib3.connectionpool:https://edwarddonner.com:443 "GET / HTTP/1.1" 200 None
DEBUG:anthropic._base_client:Request options: {'method': 'post', 'url': '/v1/messages', 'timeout': Timeout(connect=5.0, read=600, write=600, pool=600), 'files': None, 'json_data': {'max_tokens': 500, 'messages': [{'role': 'user', 'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and l

Here's a summary of Edward Donner's website in markdown:

# Edward Donner's Personal Website

## About Ed
- Co-founder and CTO of Nebula.io
- Previously founded untapt (acquired in 2021)
- Focuses on AI and LLM development
- Interests include coding, LLMs, DJing, and electronic music production

## Professional Work
- Leading Nebula.io, which applies AI to talent discovery and recruitment
- Works with proprietary LLMs specialized in talent management
- Holds patents for matching model technology
- Platform has received awards and media coverage

## Recent Posts/Announcements
1. The Complete Agentic AI Engineering Course (April 2025)
2. LLM Workshop – Hands-on with Agents – resources (January 2025)
3. Welcome, SuperDataScientists! (December 2024)
4. Mastering AI and LLM Engineering – Resources (November 2024)

## Projects
- Connect Four
- Outsmart - An LLM-based arena for testing AI diplomacy and strategy

The website serves as a professional portfolio and blog, focusing primarily on AI, LLM engineering, and talent technology, while also showcasing Ed's personal interests and achievements in the tech industry.

## Through OpenAI Library

In [20]:
ANTHROPIC_API = "https://api.anthropic.com/v1/"
anthropic_via_openai = OpenAI(base_url=ANTHROPIC_API, api_key=ANTHROPIC_API_KEY)

In [21]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [22]:
def summarize_openai(url):
    website = Website(url)
    messages = messages_for(website)
    # print(messages)
    response = anthropic_via_openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [23]:
def display_summary_openai(url):
    summary = summarize_openai(url)
    display(Markdown(summary))

In [24]:
url = "https://edwarddonner.com"
display_summary_openai(url)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): edwarddonner.com:443
DEBUG:urllib3.connectionpool:https://edwarddonner.com:443 "GET / HTTP/1.1" 200 None
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'}, {'role': 'user', 'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too

Here's a summary of Edward Donner's website in markdown:

# Edward Donner's Personal Website

## About Ed
- Co-founder and CTO of **Nebula.io**
- Previously founded AI startup untapt (acquired in 2021)
- Focuses on AI and LLM technology
- Interests include coding, LLM experimentation, DJing, and electronic music production

## Professional Work
- Leading Nebula.io, which applies AI to talent discovery and recruitment
- Works with proprietary LLMs specialized in talent management
- Holds patents for matching models
- Platform has received awards and media coverage

## Recent Posts/Announcements
1. The Complete Agentic AI Engineering Course (April 2025)
2. LLM Workshop – Hands-on with Agents – resources (January 2025)
3. Welcome, SuperDataScientists! (December 2024)
4. Mastering AI and LLM Engineering – Resources (November 2024)

## Projects
- **Connect Four**: (Project details not provided)
- **Outsmart**: An arena for LLMs to compete in diplomacy and deviousness

The website serves as a professional portfolio and blog, focusing primarily on AI, LLM engineering, and talent technology.

In [25]:
def messages_for(website):
    return {"system": system_prompt,
            "messages":[
                        {"role": "user", "content": user_prompt_for(website)}
                ]
            }

In [26]:
import requests
import json

def ask_claude(prompt, api_key, model="claude-3-sonnet-20240229", max_tokens=1024):
    url = "https://api.anthropic.com/v1/messages"
    
    headers = {
        "x-api-key": api_key,
        "anthropic-version": "2023-06-01",
        "Content-Type": "application/json"
    }

    payload = {
        "model": MODEL,
        "max_tokens": max_tokens,
        "messages": [
            {"role": "user", "content": prompt}
        ]
    }

    response = requests.post(url, headers=headers, data=json.dumps(payload))

    if response.status_code == 200:
        data = response.json()
        return data["content"][0]["text"]
    else:
        raise Exception(f"Error {response.status_code}: {response.text}")

# Example usage
if __name__ == "__main__":
    API_KEY = ANTHROPIC_API_KEY
    prompt = "Explain the concept of retrieval-augmented generation in simple terms."
    website = "https://edwarddonner.com"
    PROMPT = "Who is Max Plank?"

    try:
        answer = ask_claude(prompt=PROMPT, api_key=API_KEY)
        print("Claude says:\n", answer)
    except Exception as e:
        print(e)


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.anthropic.com:443
DEBUG:urllib3.connectionpool:https://api.anthropic.com:443 "POST /v1/messages HTTP/1.1" 200 None


Claude says:
 Max Planck (1858-1947) was a renowned German theoretical physicist who is considered one of the most important scientific figures of the 20th century. He is best known for:

1. Founding quantum theory: In 1900, he proposed that energy is emitted and absorbed in discrete packets called "quanta" rather than as a continuous wave. This revolutionary idea laid the foundation for quantum physics.

2. Planck's constant: He introduced what became known as Planck's constant (h), a fundamental physical constant that is crucial to quantum mechanics.

3. Black-body radiation: He solved the ultraviolet catastrophe problem by explaining black-body radiation through his quantum theory.

4. Nobel Prize: He was awarded the Nobel Prize in Physics in 1918 for his discovery of energy quanta.

5. Scientific legacy: He also made significant contributions to thermodynamics, statistical mechanics, and special relativity.

6. Institutional impact: He served as president of the Kaiser Wilhelm Soci